In [17]:
import json 
import requests
import time 
from openai import OpenAI
from  dotenv import dotenv_values

OPENAI_API_KEY = dotenv_values(".env").get("OPENAI_API_KEY")
FMP_API_KEY = dotenv_values(".env").get("FMP_API_KEY")

GPT_MODEL = "gpt-3.5-turbo-1106"

client:OpenAI = OpenAI(api_key=OPENAI_API_KEY)


In [18]:
def show_json(obj):
    return json.loads(obj.model_dump_json())

# Step 1: Defining Financial Functions

This is a paragraph of text. You can write your content here. You can use *italic* or **bold** formatting, create lists, and more.


## Unordered List
- Financial statements (income, balance sheet, & cash flow)
- Financial ratios
- Key metrics
- Financial growth




In [19]:
# Define financial statement functions

# AAPL
def get_income_statement(ticker,period,limit):
    url:str = f"https://financialmodelingprep.com/api/v3/income-statement/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())
   


def get_balance_sheet(ticker, period,limit):
    url:str = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())



def get_cash_flow_statement(ticker, period,limit):
    url:str = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())



def get_key_metrics(ticker, period,limit):
    url:str = f"https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())



def get_financial_ratios(ticker, period,limit):
    url:str = f"https://financialmodelingprep.com/api/v3/ratios/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())



def get_financial_growth(ticker, period,limit):
    url:str = f"https://financialmodelingprep.com/api/v3/financial-growth/{ticker}?period={period}&limit={limit}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())



In [20]:
available_functions = {
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_cash_flow_statement": get_cash_flow_statement,
    "get_key_metrics": get_key_metrics,
    "get_financial_ratios": get_financial_ratios,
    "get_financial_growth": get_financial_growth
}

In [21]:
tools=[{
        "type":"function",
         "function":{
                "name":"get_income_statement",
                "description":"get income statement",
                "parameters":{
                    "type":"object",
                    "properties":{
                        "ticker":{ "type":"string"},
                        "period":{ "type":"string"},
                        "limit":{ "type":"string"},
                    }
                }
            }
        },{
        "type":"function",
         "function":{
                "name":"get_balance_sheet",
                "description":"get balance sheet",
                "parameters":{
                    "type":"object",
                    "properties":{
                        "ticker":{ "type":"string"},
                        "period":{ "type":"string"},
                        "limit":{ "type":"string"},
                    }
                }
            }
        }]

In [22]:
from openai.types.beta.assistant import Assistant

# def run_assistant(user_message):
assistant:Assistant =  client.beta.assistants.create(
    model=GPT_MODEL,
    tools=tools,
    instructions="Act as a financial analyst by accessing financial data through the Financial Modeling Prep API. Your capabilities include analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends"
)

    # display(show_json(assistant))


In [23]:
from openai.types.beta.thread import Thread
thread:Thread = client.beta.threads.create()

In [24]:
from openai.types.beta.threads.thread_message import ThreadMessage
from openai.types.beta.threads.run import Run


def run_assistant(user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id,
        content=user_message,
        role="user"
    )

    run:Run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id
    )

    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run.status == "requires_action":
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            tool_outputs = []

            for tool_call in tool_calls:

                function_name =  tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                if function_name in available_functions:
                    function_to_call =  available_functions[function_name]
                    output = function_to_call(**function_args)

                    tool_outputs.append({
                        "tool_call_id":tool_call.id,
                        "output":output
                    })
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )           
        elif run.status == "completed":
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            for message in messages.data:
                role_label = "User" if message.role == "user" else "Assistant"
                message_content = message.content[0].text.value
                print(f"{role_label}: {message_content}\n")
            break  # Exit the loop after processing the completed run

        elif run.status == "failed":
            print("Run failed.")
            display(show_json(run))
            break

        elif run.status in ["in_progress", "queued"]:
            print(f"Run is {run.status}. Waiting...")
            time.sleep(1)  # Wait for 5 seconds before checking again

        else:
            print(f"Unexpected status: {run.status}")
            display(show_json(run))

            break





      
run_assistant("Compare the financial health of Microsoft and Apple over the last four years, get income statement ,focusing on their balance sheets and key financial ratios")


Run is in_progress. Waiting...
Run is in_progress. Waiting...
Run is in_progress. Waiting...
Run is in_progress. Waiting...
Run is in_progress. Waiting...
Run failed.


{'id': 'run_JJV4MkjgUWgvxcwpQjid71NL',
 'assistant_id': 'asst_m2A9LgNzkvkhPqJ5wI1frJxx',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1704210136,
 'expires_at': None,
 'failed_at': 1704210142,
 'file_ids': [],
 'instructions': 'Act as a financial analyst by accessing financial data through the Financial Modeling Prep API. Your capabilities include analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends',
 'last_error': {'code': 'rate_limit_exceeded',
  'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.'},
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1704210136,
 'status': 'failed',
 'thread_id': 'thread_DWrhJIGZW2XCAdOUgGji0WUj',
 'tools': [{'function': {'name': 'get_income_statement',
  